# SQLite Database Types

Checking on the types within the database to investigate connector-x compatibility as per https://github.com/sfu-db/connector-x/blob/main/Types.md#sqlite.

Example errors:
- `RuntimeError: Invalid column type Text at index: 61, name: Cytoplasm_Correlation_Costes_AGP_DNA`
- `RuntimeError: Invalid column type Text at index: 64, name: Cytoplasm_Correlation_Costes_AGP_RNA`
- `RuntimeError: Invalid column type Text at index: 74, name: Cytoplasm_Correlation_Costes_Mito_DNA`
- `...Cytoplasm_Correlation_K_ER_Mito`
- 

In [ ]:
import sqlite3

import pandas as pd

In [ ]:
# create connections for sqlite
# reference: https://nih.figshare.com/articles/dataset/Cell_Health_-_Cell_Painting_Single_Cell_Profiles/9995672
sqlite_conn = sqlite3.connect("SQ00014613.sqlite")
sqlite_err_conn = sqlite3.connect("testing_err_SQ00014613.sqlite")

In [ ]:
sql = """
PRAGMA table_info(Image);
"""
image_cols = pd.read_sql(sql, con=sqlite_conn)
image_cols["type"].value_counts()

In [ ]:
sql = """
PRAGMA table_info(Cells);
"""
cells_cols = pd.read_sql(sql, con=sqlite_conn)
cells_cols["type"].value_counts()

In [ ]:
sql = """
PRAGMA table_info(Cytoplasm);
"""
cyto_cols = pd.read_sql(sql, con=sqlite_conn)
cyto_cols["type"].value_counts()

In [ ]:
sql = """
PRAGMA table_info(Nuclei);
"""
nuclei_cols = pd.read_sql(sql, con=sqlite_conn)
nuclei_cols["type"].value_counts()

In [ ]:
df_dict = {
    "image": image_cols,
    "cells": cells_cols,
    "cytoplasm": cyto_cols,
    "nuclei": nuclei_cols,
}
len(df_dict.keys())

In [ ]:
df_dict["image"]

In [ ]:
sql = """
select * from Cytoplasm
where rowid = 61 or rowid = 60;
"""
cyto_errs = pd.read_sql(sql, con=sqlite_conn)
cyto_errs

In [ ]:
cyto_errs["Cytoplasm_Correlation_Costes_AGP_DNA"]

In [ ]:
sql = """
select ObjectNumber, 
    Cytoplasm_Correlation_Costes_AGP_DNA, 
    typeof(Cytoplasm_Correlation_Costes_AGP_DNA) from Cytoplasm
where rowid between 60 and 61;
"""
cyto_errs = pd.read_sql(sql, con=sqlite_conn)
cyto_errs

In [ ]:
cyto_cols[cyto_cols["name"] == "Cytoplasm_Correlation_Costes_AGP_DNA"]

In [ ]:
sql = """
select ObjectNumber, 
    Cytoplasm_Correlation_Costes_AGP_DNA, 
    typeof(Cytoplasm_Correlation_Costes_AGP_DNA) from Cytoplasm
where typeof(Cytoplasm_Correlation_Costes_AGP_DNA) != 'real'
and typeof(Cytoplasm_Correlation_Costes_AGP_DNA) not in ('text')
"""
cyto_errs = pd.read_sql(sql, con=sqlite_conn)
cyto_errs

In [ ]:
sql = """
select ObjectNumber, 
    Cytoplasm_Correlation_Costes_AGP_DNA, 
    typeof(Cytoplasm_Correlation_Costes_AGP_DNA) from Cytoplasm
where typeof(Cytoplasm_Correlation_Costes_AGP_DNA) != 'real'
"""
sqlite_conn.execute(sql).fetchall()[0]

In [ ]:
sql = """
select ObjectNumber, 
    Cytoplasm_Correlation_Costes_AGP_DNA,
    replace(Cytoplasm_Correlation_Costes_AGP_DNA, 'nan', NULL),
    typeof(Cytoplasm_Correlation_Costes_AGP_DNA) from Cytoplasm
where typeof(Cytoplasm_Correlation_Costes_AGP_DNA) != 'real';
"""
cyto_errs = pd.read_sql(sql, con=sqlite_conn)
cyto_errs

In [ ]:
tablename = "cytoplasm"
number_types = [
    "INT",
    "INTEGER",
    "TINYINT",
    "SMALLINT",
    "MEDIUMINT",
    "BIGINT",
    "UNSIGNED BIG INT",
    "INT2",
    "INT8",
    "REAL",
    "DOUBLE",
    "DOUBLE PRECISION",
    "FLOAT",
    "NUMERIC",
    "DECIMAL",
    "BOOLEAN",
]
text_types = [
    "CHARACTER",
    "VARCHAR",
    "VARYING CHARACTER",
    "NCHAR",
    "NATIVE CHARACTER",
    "NVARCHAR",
    "TEXT",
    "CLOB",
]
number_types_str_upper = ",".join([f"'{name}'" for name in number_types])

text_types_str_lower = ",".join(
    [f"'{name}'" for name in [name.lower() for name in text_types]]
)
sql = (
    f"SELECT name, type FROM PRAGMA_TABLE_INFO('{tablename}')"
    f" where type in ({number_types_str_upper})"
)
col_result = pd.read_sql(sql, con=sqlite_err_conn)

filter_query = f"select * from {tablename}"
for col in col_result["name"].values.tolist():
    if col == col_result["name"].iloc[0]:
        filter_query += " where "
    filter_query += f"typeof({col}) not in ({text_types_str_lower})"
    if col != col_result["name"].iloc[-1]:
        filter_query += " and "
filter_query

In [ ]:
query = pd.read_sql(filter_query, con=sqlite_err_conn)
query[["Cytoplasm_Correlation_Costes_AGP_DNA", "typeof"]]

In [ ]:
query["ImageNumber"].info()

In [ ]:
for tabname, df in df_dict.items():
    for colname in df[df["type"].isin(["FLOAT", "BIGINT"])]["name"].values.tolist():
        sql = f"UPDATE {tabname} SET {colname} = replace({colname}, 'nan', 0);"
        sqlite_conn.execute(sql)
        sqlite_conn.commit()